# Text tokenizers

In [ ]:
#| default_exp text.tokenizers

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

In [ ]:
#| export

# torch
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.optim import SGD

# torchtext
import torchtext
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer
from torchtext.datasets import AG_NEWS

# hf
import datasets
from transformers import AutoTokenizer, DataCollatorForLanguageModeling

# data 
import pandas as pd
import numpy as np

# ui
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

# python
from typing import Dict, List, Tuple, Optional, Set, Iterable
from collections import Counter, OrderedDict
from dataclasses import dataclass, asdict
from plum import dispatch

# nimrod
# from nimrod.models.lm import Vocab

/opt/bitnami/python/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Torchtext
https://pytorch.org/text/0.16.0/data_utils.html

In [ ]:
#| export 

class Tokenizer:
    def __init__(self,
                backend:str='spacy', # backend tokenizer default to spacy
                language:str='en', # language on which tokenization is applied
                bos:bool=False, # add beginning of sentence tag <bos>
                eos:bool=False, # add end of sentence tag <eos>
                ):
        if backend == 'spacy' and language == 'en':
            language = 'en_core_web_sm'
        if backend== 'character_based':
            self.tokenizer = self.character_tokenizer
        else:
            self.tokenizer = get_tokenizer(backend, language=language)
        self.bos = bos
        self.eos = eos
        self.backend = backend
        print(f"# Tokenizer uses {self.backend} backend")
    
    @staticmethod
    def character_tokenizer(text:str)->List[str]:
        return [c for c in text]
    
    @dispatch
    def __call__(self, text:str)->List[str]:
        res = self.tokenizer(text)
        if self.bos:
            res = ['<bos>'] + res
        if self.eos:
            res = res + ['<eos>']
        return(res)
    
    @dispatch
    def __call__(self, texts:List[str])->List[List[str]]:
        return [self(text) for text in texts]
    
    @dispatch # to replace Iterable
    # works with agnews type of dataset [(index, text)]
    def __call__(self, data_iter:Iterable)->Iterable:
        for _, text in data_iter:
            yield self(text)

    @dispatch    
    def inverse(self, tokens:List[str])->str:
        if self.backend == 'character_based':
            return ''.join(tokens)
        # TODO: take care of white spaces
        else:
            return ' '.join(tokens)

    @dispatch
    def inverse(self, list_of_tokens:List[List[str]])->List[str]:
        s = []
        for tokens in list_of_tokens:
            s.append(self.inverse(tokens))
        return s

### Usage

#### String

In [ ]:
tok = Tokenizer(backend='character_based', bos=True, eos=True)
# str -> List[str]
s = "Oh, yeah I'm not sure..."
tokenized = tok(s)
print("original sentence: ", s)
print("tokenized: ", tokenized)
print("un-tokenized: ", tok.inverse(tokenized))

tok = Tokenizer(backend='spacy', bos=True, eos=True)
# str -> List[str]
s = "Oh, yeah I'm not sure..."
tokenized = tok(s)
print("original sentence: ", s)
print("tokenized: ", tokenized)
print("un-tokenized: ", tok.inverse(tokenized))

tok = Tokenizer(backend='basic_english', bos=True, eos=True)
# str -> List[str]
s = "Oh, yeah I'm not sure..."
tokenized = tok(s)
print("original sentence: ", s)
print("tokenized: ", tokenized)
print("un-tokenized: ", tok.inverse(tokenized))

# Tokenizer uses character_based backend
original sentence:  Oh, yeah I'm not sure...
tokenized:  ['<bos>', 'O', 'h', ',', ' ', 'y', 'e', 'a', 'h', ' ', 'I', "'", 'm', ' ', 'n', 'o', 't', ' ', 's', 'u', 'r', 'e', '.', '.', '.', '<eos>']
un-tokenized:  <bos>Oh, yeah I'm not sure...<eos>
# Tokenizer uses spacy backend
original sentence:  Oh, yeah I'm not sure...
tokenized:  ['<bos>', 'Oh', ',', 'yeah', 'I', "'m", 'not', 'sure', '...', '<eos>']
un-tokenized:  <bos> Oh , yeah I 'm not sure ... <eos>
# Tokenizer uses basic_english backend
original sentence:  Oh, yeah I'm not sure...
tokenized:  ['<bos>', 'oh', ',', 'yeah', 'i', "'", 'm', 'not', 'sure', '.', '.', '.', '<eos>']
un-tokenized:  <bos> oh , yeah i ' m not sure . . . <eos>


#### List of strings

In [ ]:
# List[str]->List[List[str]]
s = ["Oh, yeah I don't know dude...", "this is a test"]
tokenized = tok(s)
print("original sentence: ", s)
print("tokenized: ", tokenized)
print("un-tokenized: ", tok.inverse(tokenized))

original sentence:  ["Oh, yeah I don't know dude...", 'this is a test']
tokenized:  [['<bos>', 'oh', ',', 'yeah', 'i', 'don', "'", 't', 'know', 'dude', '.', '.', '.', '<eos>'], ['<bos>', 'this', 'is', 'a', 'test', '<eos>']]
un-tokenized:  ["<bos> oh , yeah i don ' t know dude . . . <eos>", '<bos> this is a test <eos>']


#### Iterable

In [ ]:
# Iterable -> Iterable
tok = Tokenizer()
ds = AG_NEWS(split='test') # data pipe
sample = next(iter(ds)) # (label, text)
print(sample)
it = tok(ds)
tokens = [token for token in it]
print(len(tokens))
print(tokens[:2])

# Tokenizer uses spacy backend
(3, "Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.")


/opt/bitnami/python/lib/python3.11/site-packages/torch/utils/data/datapipes/iter/combining.py:297: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


7600
[['Fears', 'for', 'T', 'N', 'pension', 'after', 'talks', 'Unions', 'representing', 'workers', 'at', 'Turner', '  ', 'Newall', 'say', 'they', 'are', "'", 'disappointed', "'", 'after', 'talks', 'with', 'stricken', 'parent', 'firm', 'Federal', 'Mogul', '.'], ['The', 'Race', 'is', 'On', ':', 'Second', 'Private', 'Team', 'Sets', 'Launch', 'Date', 'for', 'Human', 'Spaceflight', '(', 'SPACE.com', ')', 'SPACE.com', '-', 'TORONTO', ',', 'Canada', '--', 'A', 'second\\team', 'of', 'rocketeers', 'competing', 'for', 'the', ' ', '#', '36;10', 'million', 'Ansari', 'X', 'Prize', ',', 'a', 'contest', 'for\\privately', 'funded', 'suborbital', 'space', 'flight', ',', 'has', 'officially', 'announced', 'the', 'first\\launch', 'date', 'for', 'its', 'manned', 'rocket', '.']]


# Numericalizer
https://pytorch.org/text/stable/vocab.html

In [ ]:
#| export
# TODO: add more special characters
class Numericalizer():
    def __init__(self, tokens_iter:Iterable, specials=["<pad>", "<unk>", "<bos>", "<eos>"]):
        self._vocab = self.build_map_from_iter(tokens_iter, specials)
    
    def build_map_from_iter(self,data_iter:Iterable, specials=None):
        self._vocab = torchtext.vocab.build_vocab_from_iterator(data_iter, specials=specials)
        if "<unk>" in specials:
            self._vocab.set_default_index(self._vocab["<unk>"])
        return self._vocab

    @dispatch
    def __call__(self, texts:List[str])->List[List[int]]:
        # TODO: check self._vocab has been built
        return [self._vocab[text] for text in texts]
    
    @dispatch
    def __call__(self, texts:List[List[str]]):
        # TODO: use nested list comprehension
        res = []
        for row in texts:
            res.append([self._vocab[text] for text in row])
        return res
        
    @dispatch
    def __call__(self, text:str)->int:
        return self._vocab[text]
    
    @property
    def vocab(self):
        return(self._vocab)
    
    @dispatch
    def inverse(self, idx:int)->str:
        return self._vocab.get_itos()[idx]

    @dispatch
    def inverse(self, indices:List[int])->List[str]:
        return [self._vocab.get_itos()[i] for i in indices]

### Usage

In [ ]:
tok = Tokenizer()
# In the case of agnews, dataset is: [(index, text)]
def token_iterator(data_iter:Iterable)->Iterable:
    for _, text in data_iter:
        yield tok(text)
tok_it= token_iterator(ds)
# initialize numericalizer based on token iterator
num = Numericalizer(tok_it)

# Tokenizer uses spacy backend


In [ ]:
print(num('<pad>'), num('<unk>'))

0 1


In [ ]:
print(num.vocab['the'])
print(num('the'))
print(num(['<bos>', '<pad>', '<unk>', 'a', 'this', 'the', 'lkjsdf']))
print(num.inverse(0))
print(num.inverse([6,55]))
print(num([['<bos>', '<pad>'], ['<unk>', 'a', 'this', 'the', 'lkjsdf']]))

4
4
[2, 0, 1, 9, 58, 4, 1]
<pad>
['.', 'Monday']
[[2, 0], [1, 9, 58, 4, 1]]


In [ ]:
tokens = tok(["here we go. asdflkj", "it was time..."])
print(tokens)
print([num(tok) for tok in tokens])
print(num(tokens))

[['here', 'we', 'go', '.', 'asdflkj'], ['it', 'was', 'time', '...']]
[[534, 1040, 310, 6, 1], [34, 40, 101, 67]]
[[534, 1040, 310, 6, 1], [34, 40, 101, 67]]


## Hugging Face tokenizers
https://huggingface.co/learn/nlp-course/chapter2/4?fw=pt

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

/opt/bitnami/python/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)
print(tokens)

['Using', 'Ġa', 'ĠTrans', 'former', 'Ġnetwork', 'Ġis', 'Ġsimple']


In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[12814, 257, 3602, 16354, 3127, 318, 2829]


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()